In [6]:
%matplotlib inline

In [7]:
import datetime, time
import lxml

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [9]:
seqs = pd.read_csv("../data/sequential_id.csv")

In [10]:
seqs['name'] = seqs.name.str.upper()
seqs['name'] = seqs.name.replace('AURÉLIO NOMURA', 'AURELIO NOMURA')
seqs['name'] = seqs.name.replace('CAIO MIRANDA CARNEIRO', 'CAIO MIRANDA')
seqs['name'] = seqs.name.replace('ANTONIO DONATO', 'DONATO')
seqs['name'] = seqs.name.replace('EDUARDO SUPLICY', 'EDUARDO MATARAZZO SUPLICY') 
seqs['name'] = seqs.name.replace('MARIO COVAS NETO', 'MÁRIO COVAS NETO')
seqs['name'] = seqs.name.replace('REGINALDO TRIPOLI', 'REGINALDO TRÍPOLI')
seqs['name'] = seqs.name.replace('ZÉ TURIN', 'ZÉ TURIM')

In [11]:
xmls = [] 
xmls.append('http://www2.camara.sp.gov.br/sisgv/Arquivos/201705.XML')

In [12]:
lista_gastos=[]
for link in xmls:
    r = requests.get(link)
    xml = BeautifulSoup(r.content, 'xml')
    gastos = xml.find_all('TabelaPortalITEMREEMBOLSO')
    for s in gastos:
        chave = s.Chave.text
        centro_custos = s.CENTROCUSTOSID.text
        departamento = s.DEPARTAMENTO.text
        tipodep = s.TIPODEDEPARTAMENTO
        vereador = s.VEREADOR.text
        ano = s.ANO.text
        mes = s.MES.text
        despesa = s.DESPESA.text
        cnpj = s.CNPJ.text
        fornecedor = s.FORNECEDOR.text
        valor = s.VALOR.text
        data = datetime.date(int(ano), int(mes), 1)

        dict_gastos = {'DATA': data, 'Chave':chave, 'CENTROCUSTOSID': centro_custos, 'DEPARTAMENTO': departamento, 
                'TIPO_DEPARTAMENTO':tipodep, 'VEREADOR': vereador, 'DESPESA':despesa, 'CNPJ': cnpj, 
                       'FORNECEDOR': fornecedor, 'VALOR': valor}
        lista_gastos.append(dict_gastos)

In [13]:
df = pd.DataFrame().from_records(lista_gastos)

In [14]:
df = df[['DATA', 'VEREADOR', 'DEPARTAMENTO', 'DESPESA', 'VALOR', 'FORNECEDOR', 'CNPJ']]
df[['DATA']] = df[['DATA']].apply(pd.to_datetime)
df[['VALOR']] = df[['VALOR']].apply(pd.to_numeric)

In [15]:
df = pd.merge(df, seqs, how='left', left_on='VEREADOR', right_on='name')

In [18]:
df = df.drop(['VEREADOR', 'name'], axis=1)

In [20]:
df = df[df.sequential_id.notnull()]

In [21]:
df = df.rename(columns={
    'DATA': 'date', 
    'VEREADOR': 'candidate',
    'DEPARTAMENTO': 'department', 
    'DESPESA': 'expense',
    'VALOR': 'value', 
    'FORNECEDOR': 'provider', 
    'CNPJ': 'cnpj',
})

In [24]:
df.head()

,date,department,expense,value,provider,cnpj,sequential_id
0,2017-01-01,GABINETE DE VEREADOR,COMBUSTIVEL,880.00,PROTOTIPO AUTO POSTO LTDA.,43.900.851/0001-01,2.500000e+11
1,2017-01-01,GABINETE DE VEREADOR,INTERMEDIADO - CORREIOS,5993.27,CAMARA MUNICIPAL DE SÃO PAULO,50.176.288/0001-28,2.500000e+11
2,2017-01-01,GABINETE DE VEREADOR,MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE C...,219.45,CENTER PAPEIS COMERCIAL LTDA.,06.226.820/0001-82,2.500000e+11
3,2017-01-01,GABINETE DE VEREADOR,TELEFONE MOVEL,266.10,TELEFONICA BRASIL S/A,02.558.157/0001-62,2.500000e+11
4,2017-02-01,GABINETE DE VEREADOR,COMBUSTIVEL,150.00,POSTO DE SERVIÇOS 23 DE MAIO LTDA.,18.811.994/0001-91,2.500000e+11


In [25]:
today = datetime.date.today()
df.to_csv(f"../data/{today}-expenses.csv", index=False)